In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.preprocessing import OrdinalEncoder
from category_encoders import HashingEncoder

In [2]:
non_laundering_df = pd.read_csv("part_1.csv")
laundering_df = pd.read_csv("laundering_transactions.csv")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_5964\3595299628.py:1: DtypeWarning: Columns (5,7) have mixed types. Specify dtype option on import or set low_memory=False.
  non_laundering_df = pd.read_csv("part_1.csv")


In [33]:
non_laundering_df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2019/01/23 00:53,822E8A320,85A4AF500,887BC7120,A260915B0,41.05,Euro,41.05,Euro,Credit Card,0
1,2019/01/03 03:23,920F27020,926C36820,91D1A4620,93B0685F0,9318.37,Ruble,9318.37,Ruble,Cash,0
2,2019/01/21 09:29,B189DCBE0,B3199B700,B1CC75EA0,B328C2650,0.139284,Bitcoin,0.139284,Bitcoin,Cash,0
3,2019/01/05 00:14,929ED0080,99EC3E540,964A86860,A5EED4620,8.17,Euro,8.17,Euro,Credit Card,0
4,2019/01/02 03:56,873813880,9516363D0,97AAEFB30,A01FE3050,978.56,US Dollar,978.56,US Dollar,Cheque,0


In [34]:
laundering_df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
0,2019/01/01 00:05,8021ADE00,80238F220,9A7F59FA0,A23691240,33.97,US Dollar,33.97,US Dollar,Credit Card,1
1,2019/01/01 00:28,80272F020,81DE06080,84461DF90,88BC68FF0,75.32,US Dollar,75.32,US Dollar,Credit Card,1
2,2019/01/01 00:13,81F65EF20,82B6F8380,A68E33DF0,A755CFD50,144.84,US Dollar,144.84,US Dollar,Credit Card,1
3,2019/01/01 00:24,829EF5F10,82B6F8470,8BC205620,90B78CF60,54.24,US Dollar,54.24,US Dollar,Credit Card,1
4,2019/01/01 00:08,8321EF9D0,860996AB0,8001B5DA0,802477980,39.52,US Dollar,39.52,US Dollar,Credit Card,1


In [6]:
laundering_df.shape, non_laundering_df.shape

((55423, 11), (120000, 11))

In [5]:
non_laundering_df = non_laundering_df.sample(n=120000)

In [7]:
total_df = pd.concat([laundering_df, non_laundering_df], axis=0)

In [8]:
total_df.shape

(175423, 11)

In [9]:
# Shuffle the dataset
df = total_df.sample(frac=1, random_state=42)
df.head()

,Timestamp,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering
721374,2019/01/18 22:28,AEA7127E0,B001D8FF0,AE1862530,B0020F8C0,2735.57,Saudi Riyal,2735.57,Saudi Riyal,ACH,0
1731318,2019/01/22 01:29,A4FA712A0,A5230B5F0,865A7E920,A5230B080,1497.83,Euro,1497.83,Euro,ACH,0
1363364,2019/01/28 18:12,A75670510,A75F51B90,A7855FEE0,A7C491440,767.62,Swiss Franc,767.62,Swiss Franc,Cheque,0
448157,2019/01/08 00:29,985D78EA0,9A6EBFB00,85819CDF0,8B7802F80,85.73,US Dollar,85.73,US Dollar,Credit Card,0
788971,2019/01/03 12:20,9AA836F10,9C11F0E70,9CBBBFCB0,9D0567C30,9705.26,Australian Dollar,9705.26,Australian Dollar,ACH,0


In [39]:
# df.to_csv("shuffled_dataset(part_1+launderingdata).csv", index=False)

In [40]:
df.columns

Index(['Timestamp', 'From Bank', 'Account', 'To Bank', 'Account.1',
       'Amount Received', 'Receiving Currency', 'Amount Paid',
       'Payment Currency', 'Payment Format', 'Is Laundering'],
      dtype='object')

In [41]:
for col in df.columns:
    df[col] = df[col].replace("(null)", np.nan)

Checking for NULL value

In [42]:
df.isnull().sum(axis = 0)

Timestamp               0
From Bank               0
Account                 0
To Bank                 0
Account.1               0
Amount Received       146
Receiving Currency      0
Amount Paid            25
Payment Currency        0
Payment Format          0
Is Laundering           0
dtype: int64

In [43]:
df["Amount Received"] = df["Amount Received"].astype(float)
df["Amount Paid"] = df["Amount Paid"].astype(float)

In [44]:
# Filling up the null value of Amount Received column based on the value of Amount Paid.
for i in range(len(df["Amount Received"])):
    if pd.isnull(df["Amount Received"].iloc[i]) and (not pd.isnull(df["Amount Paid"].iloc[i])):
        df["Amount Received"].iloc[i] = df["Amount Paid"].iloc[i]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15116\3633458371.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Amount Received"].iloc[i] = df["Amount Paid"].iloc[i]


In [45]:
# Filling up the null value of Amount Paid column based on the value of Amount Received.
for i in range(len(df["Amount Paid"])):
    if pd.isnull(df["Amount Paid"].iloc[i]) and (not pd.isnull(df["Amount Received"].iloc[i])):
        df["Amount Paid"].iloc[i] = df["Amount Received"].iloc[i]

C:\Users\Administrator\AppData\Local\Temp\ipykernel_15116\2356711384.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Amount Paid"].iloc[i] = df["Amount Received"].iloc[i]


In [46]:
df.isnull().sum(axis = 0)

Timestamp             0
From Bank             0
Account               0
To Bank               0
Account.1             0
Amount Received       0
Receiving Currency    0
Amount Paid           0
Payment Currency      0
Payment Format        0
Is Laundering         0
dtype: int64

In [47]:
if df.isnull().any().any():
    print("At least one column contains null values.")
else:
    print("No column contains null values.")

No column contains null values.


In [48]:
# Convert the timestamp column to datetime objects
df["Timestamp"] = df["Timestamp"].apply(lambda x: datetime.strptime(x, "%Y/%m/%d %H:%M"))

# Extract year, month, day, hour, and minute components into separate columns
df["year"] = df["Timestamp"].dt.year
df["month"] = df["Timestamp"].dt.month
df["day"] = df["Timestamp"].dt.day
df["hour"] = df["Timestamp"].dt.hour
df["minute"] = df["Timestamp"].dt.minute

In [49]:
df = df.drop("Timestamp", axis=1)

In [51]:
# Zero mean and a standard deviation of 1
quant_features = ['Amount Received', 'Amount Paid']
scaled_features = {}

for each in quant_features:
    mean, std = df[each].mean(), df[each].std()
    scaled_features[each] = [mean, std]
    df.loc[:, each] = (df[each] - mean)/std

In [52]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2322999 entries, 2212990 to 2163687
Data columns (total 15 columns):
 #   Column              Dtype  
---  ------              -----  
 0   From Bank           object 
 1   Account             object 
 2   To Bank             object 
 3   Account.1           object 
 4   Amount Received     float64
 5   Receiving Currency  object 
 6   Amount Paid         float64
 7   Payment Currency    object 
 8   Payment Format      object 
 9   Is Laundering       int64  
 10  year                int64  
 11  month               int64  
 12  day                 int64  
 13  hour                int64  
 14  minute              int64  
dtypes: float64(2), int64(6), object(7)
memory usage: 283.6+ MB


In [53]:
df.count()

From Bank             2322999
Account               2322999
To Bank               2322999
Account.1             2322999
Amount Received       2322999
Receiving Currency    2322999
Amount Paid           2322999
Payment Currency      2322999
Payment Format        2322999
Is Laundering         2322999
year                  2322999
month                 2322999
day                   2322999
hour                  2322999
minute                2322999
dtype: int64

In [54]:
columns = ["Payment Format", "Payment Currency", "Receiving Currency", "Is Laundering", "year", "month", 'day', 'hour', 'minute']

print('------------------------------------------------------')
for col in columns:
    print(col)
    print(df[col].value_counts())
    print('------------------------------------------------------\n\n')


------------------------------------------------------
Payment Format
Credit Card     751250
ACH             568303
Reinvestment    537679
Cheque          348426
Cash             78933
Wire             33769
Bitcoin           4639
Name: Payment Format, dtype: int64
------------------------------------------------------


Payment Currency
US Dollar            849699
Euro                 535979
Yuan                 136364
Bitcoin              100674
Yen                   77600
Rupee                 69439
Mexican Peso          68787
Swiss Franc           68505
Ruble                 64727
UK Pound              64510
Canadian Dollar       62650
Australian Dollar     60649
Saudi Riyal           59489
Brazil Real           52196
Shekel                51731
Name: Payment Currency, dtype: int64
------------------------------------------------------


Receiving Currency
US Dollar            847754
Euro                 537367
Yuan                 134431
Bitcoin              101013
Yen            

In [55]:
df.columns

Index(['From Bank', 'Account', 'To Bank', 'Account.1', 'Amount Received',
       'Receiving Currency', 'Amount Paid', 'Payment Currency',
       'Payment Format', 'Is Laundering', 'year', 'month', 'day', 'hour',
       'minute'],
      dtype='object')

In [56]:
for col in df.columns:
    print(f"{col}: {len(df[col].unique())}")

From Bank: 98431
Account: 1779224
To Bank: 54116
Account.1: 1831709
Amount Received: 844356
Receiving Currency: 15
Amount Paid: 845301
Payment Currency: 15
Payment Format: 7
Is Laundering: 2
year: 1
month: 1
day: 31
hour: 24
minute: 60


In [57]:
df.head()

,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,year,month,day,hour,minute
2212990,806F22180,809A7E720,824E879B0,8AC7E77F0,-0.028060,US Dollar,-0.027001,US Dollar,Credit Card,0,2019,1,2,6,24
182129,976E96400,9B5DE83F0,82A31B220,862E00630,-0.028098,Euro,-0.027039,Euro,Credit Card,0,2019,1,27,0,17
1023159,817A3E5E0,9A51AF550,817A3E5E0,9A51AF550,-0.028051,US Dollar,-0.026993,US Dollar,Reinvestment,0,2019,1,31,0,9
756674,800114480,98FC49A60,990E86CF0,99EBC14D0,-0.027848,Canadian Dollar,-0.026795,Canadian Dollar,Cash,0,2019,1,3,9,25
481467,80CF261D0,822B315C0,80A5263F0,85A330490,-0.027758,US Dollar,-0.026708,US Dollar,Cheque,0,2019,1,2,13,36


In [58]:
cols_to_encode = ["From Bank", "To Bank"]
# Create an instance of the OrdinalEncoder class
encoder = OrdinalEncoder()
encoder.fit(df[cols_to_encode])

OrdinalEncoder()

In [59]:
df[cols_to_encode] = encoder.transform(df[cols_to_encode])

In [60]:
df.head()

,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,year,month,day,hour,minute
2212990,1591.0,809A7E720,4276.0,8AC7E77F0,-0.028060,US Dollar,-0.027001,US Dollar,Credit Card,0,2019,1,2,6,24
182129,40421.0,9B5DE83F0,4793.0,862E00630,-0.028098,Euro,-0.027039,Euro,Credit Card,0,2019,1,27,0,17
1023159,4398.0,9A51AF550,2942.0,9A51AF550,-0.028051,US Dollar,-0.026993,US Dollar,Reinvestment,0,2019,1,31,0,9
756674,54.0,98FC49A60,29340.0,99EBC14D0,-0.027848,Canadian Dollar,-0.026795,Canadian Dollar,Cash,0,2019,1,3,9,25
481467,2639.0,822B315C0,1437.0,85A330490,-0.027758,US Dollar,-0.026708,US Dollar,Cheque,0,2019,1,2,13,36


In [61]:
df["From Bank"] = df["From Bank"].astype(int)
df["To Bank"] = df["To Bank"].astype(int)

In [62]:
df.head()

,From Bank,Account,To Bank,Account.1,Amount Received,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,year,month,day,hour,minute
2212990,1591,809A7E720,4276,8AC7E77F0,-0.028060,US Dollar,-0.027001,US Dollar,Credit Card,0,2019,1,2,6,24
182129,40421,9B5DE83F0,4793,862E00630,-0.028098,Euro,-0.027039,Euro,Credit Card,0,2019,1,27,0,17
1023159,4398,9A51AF550,2942,9A51AF550,-0.028051,US Dollar,-0.026993,US Dollar,Reinvestment,0,2019,1,31,0,9
756674,54,98FC49A60,29340,99EBC14D0,-0.027848,Canadian Dollar,-0.026795,Canadian Dollar,Cash,0,2019,1,3,9,25
481467,2639,822B315C0,1437,85A330490,-0.027758,US Dollar,-0.026708,US Dollar,Cheque,0,2019,1,2,13,36


In [63]:
cols_to_encode = ["Account", "Account.1"]
# Create an instance of the HashingEncoder class
encoder = HashingEncoder(cols=cols_to_encode)
df_encoded = encoder.fit_transform(df)

In [64]:
df_encoded.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,From Bank,To Bank,...,Receiving Currency,Amount Paid,Payment Currency,Payment Format,Is Laundering,year,month,day,hour,minute
2212990,0,0,0,1,0,1,0,0,1591,4276,...,US Dollar,-0.027001,US Dollar,Credit Card,0,2019,1,2,6,24
182129,0,1,0,1,0,0,0,0,40421,4793,...,Euro,-0.027039,Euro,Credit Card,0,2019,1,27,0,17
1023159,0,2,0,0,0,0,0,0,4398,2942,...,US Dollar,-0.026993,US Dollar,Reinvestment,0,2019,1,31,0,9
756674,1,0,0,0,1,0,0,0,54,29340,...,Canadian Dollar,-0.026795,Canadian Dollar,Cash,0,2019,1,3,9,25
481467,0,1,0,0,1,0,0,0,2639,1437,...,US Dollar,-0.026708,US Dollar,Cheque,0,2019,1,2,13,36


In [65]:
df_encoded = df_encoded.drop("year", axis=1)
df_encoded = df_encoded.drop("month", axis=1)

In [66]:
df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2322999 entries, 2212990 to 2163687
Data columns (total 19 columns):
 #   Column              Dtype  
---  ------              -----  
 0   col_0               int64  
 1   col_1               int64  
 2   col_2               int64  
 3   col_3               int64  
 4   col_4               int64  
 5   col_5               int64  
 6   col_6               int64  
 7   col_7               int64  
 8   From Bank           int32  
 9   To Bank             int32  
 10  Amount Received     float64
 11  Receiving Currency  object 
 12  Amount Paid         float64
 13  Payment Currency    object 
 14  Payment Format      object 
 15  Is Laundering       int64  
 16  day                 int64  
 17  hour                int64  
 18  minute              int64  
dtypes: float64(2), int32(2), int64(12), object(3)
memory usage: 336.7+ MB


In [67]:
df_encoded.shape

(2322999, 19)

In [68]:
cols_to_encode = ["Receiving Currency", "Payment Currency", "Payment Format"]

new_df_encoded = pd.get_dummies(df_encoded, columns=cols_to_encode)
new_df_encoded.head()

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,From Bank,To Bank,...,Payment Currency_US Dollar,Payment Currency_Yen,Payment Currency_Yuan,Payment Format_ACH,Payment Format_Bitcoin,Payment Format_Cash,Payment Format_Cheque,Payment Format_Credit Card,Payment Format_Reinvestment,Payment Format_Wire
2212990,0,0,0,1,0,1,0,0,1591,4276,...,1,0,0,0,0,0,0,1,0,0
182129,0,1,0,1,0,0,0,0,40421,4793,...,0,0,0,0,0,0,0,1,0,0
1023159,0,2,0,0,0,0,0,0,4398,2942,...,1,0,0,0,0,0,0,0,1,0
756674,1,0,0,0,1,0,0,0,54,29340,...,0,0,0,0,0,1,0,0,0,0
481467,0,1,0,0,1,0,0,0,2639,1437,...,1,0,0,0,0,0,1,0,0,0


In [69]:
new_df_encoded.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2322999 entries, 2212990 to 2163687
Data columns (total 53 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   col_0                                 int64  
 1   col_1                                 int64  
 2   col_2                                 int64  
 3   col_3                                 int64  
 4   col_4                                 int64  
 5   col_5                                 int64  
 6   col_6                                 int64  
 7   col_7                                 int64  
 8   From Bank                             int32  
 9   To Bank                               int32  
 10  Amount Received                       float64
 11  Amount Paid                           float64
 12  Is Laundering                         int64  
 13  day                                   int64  
 14  hour                                  int64  
 15  minute   

In [70]:
new_df_encoded.to_csv("preprocessedData1.csv", index=False)